In [1]:
import ipywidgets as widgets
from IPython.display import display

In [2]:
import numpy as np
import pandas as pd
import requests as r
import re

# GUI : Input researcher name or ORCID

In [16]:
researcher = widgets.Text(value="Enter Researcher's name or ORCID") #
display(researcher)

Text(value="Enter Researcher's name or ORCID")

In [21]:
get_open_data_scores_for_researcher(researcher.value)

Fetching datasets from Zenodo
total_nr_of_hits: 14 (1 pages)
(Not yet fetching other DBs and Publications for name)


Researcher: Max Bergmann

Aggregated Openness Score: 
0.0

Usage Metrics: 
   Avg Downloads: 38.5
   Average Unique Views: 10.4


# Underlying main functions 

In [13]:
from researcher_to_data import api_zenodo
from researcher_to_publications import scholar_api
from researcher_to_publications.crossref_api import crossref_main
from publication_to_data import webscrape_datasets_from_publication

def get_datasets_of_researcher(researcher_name_or_orcid):
    """ Fetches Publications and data repositories, 
        Returns list of dataset (IN WHICHEVER FORMAT IVAN NEEDS!)"""
    
    datasets = []
    publications = []
    
    if researcher_name_or_orcid[0].isdigit():
        orcid = researcher_name_or_orcid
    
        print("Fetching datasets from Zenodo")
        datasets += api_zenodo.get_data_for_orcid(orcid)

        #print("Fetching datasets from Figshare")
        # 

        #print("Fetching datasets from DataVerse")
        #
        
        print("Fetching publications from CrossRef")
        #publications += crossref_main.get_dois(orcid)
        print("Fetching datasets from identified publications")
        print("(not functional yet)")
        #for pub in publications:
            ## Doesn't work.
            #datasets += webscrape_datasets_from_publication.get_datasets_from_publication(pub)
            #pass
        
    
    else:
        name = researcher_name_or_orcid
        name = re.sub(" ", " AND ", name)
        
        print("Fetching datasets from Zenodo")
        datasets += api_zenodo.get_data_for_name(name)
        
        print("(Not yet fetching other DBs and Publications for name)")
        #

        #print("Fetching datasets from DataVerse")
        #
        
        #print("Fetching publications from Google Scholar")
        #publications += scholar_api.main_scholar.get_publications_from_researcher(name)
        
        # print("Fetching publications from Scopus")
        # publications += ...
        
        # print("Fetching publications from WoS")
        # publications += ...
        
        
        
        #print("Fetching datasets from identified publications")
        #for pub in publications:
            ## Doesn't work.
            ## datasets += webscrape_datasets_from_publication.get_datasets_from_publication(pub)
            #pass
        
        ###
        ### Remove any duplicated in datasets !!
        ### TO DO 
        ###
        
    return datasets
        

In [12]:
import data_to_metrics.data_to_metrics as dtm

def get_open_data_scores_for_researcher(researcher_name_or_orcid,
                                        print_nb=True):
    """Returns Open Data Score for a reseacher"""
    
    datasets = get_datasets_of_researcher(researcher_name_or_orcid)
    
    scores = dtm.datasets_to_scores(datasets)
    
    if print_nb:
        print('\n\nResearcher: '+researcher_name_or_orcid)
        print('\nAggregated Openness Score: ')
        print(scores['openness_score'])
        
        print('\nUsage Metrics: ')
        print('   Avg Downloads: '+str(scores['avg_downloads']))
        print('   Average Unique Views: '+str(scores['avg_unique_views']))
    